Determine whether to use the GradientBoostingRegressor from sklearn or a Neural Network.
Evaluate which model gets better results

In [26]:
import csv
import numpy as np
import scipy
import networkx as nx
import math
import sys
import os
import subprocess
import scipy.stats
import scipy.optimize
import operator
from sklearn.manifold import TSNE
from sklearn.ensemble import GradientBoostingRegressor
import matplotlib.pyplot as plt

sys.path.append("../Python_code") # go to parent dir
from canvas_vis import * 
from analytics_combined import *
from generate_proj_to_remove import *
from project_data_analysis import *
from user_embedding import *
from segmentation import *
from evaluation import *
from nonlinear_regressor import *
import pickle
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [27]:
#Determine the size of the canvas that is being looked at
min_x = 450
max_x = 550
min_y = 450
max_y = 550


In [28]:
projects_to_remove = get_list_of_removed_proj(output_filename = "../data/proj_to_remove.txt")

input_file= "../data/sorted_tile_placements_proj.csv"
js_filename = "../data/atlas_complete.json"

names, descriptions = read_picture_names_and_descriptions(js_filename)

In [29]:
file_prefix = "10x10"

In [30]:

#TODO: How far two vertices should be to be connected (1-4)?
G, ups = create_graph(input_file, projects_to_remove, 4, min_x, max_x, min_y, max_y, file_prefix="comparison")

print("num edges = ", G.n_edges)

num edges =  12274043


In [31]:
pfile = open(file_prefix + 'graph.pkl', 'wb')
pickle.dump(G, pfile)
pfile.close()

pfile = open(file_prefix + 'ups.pkl', 'wb')
pickle.dump(ups, pfile)
pfile.close()

In [32]:
pfile = open(file_prefix + 'graph.pkl', 'rb')
G = pickle.load(pfile)
pfile.close()

pfile = open(file_prefix + 'ups.pkl', 'rb')
ups = pickle.load(pfile)
pfile.close()

In [33]:
#Defining 7 edge features and computing the information they require
#Adding a new feature without changing the rest of the code should
#be easy.
#TODO: Are there other features that would improve the segmentation?
#TODO: How many dimensions we need?

def different_color(i, j, ups, data=None):
    if ups[i][4] == ups[j][4]:
        return 0
    else:
        return 1
        
def distance_space(i, j, ups, data=None):
    xi = ups[i][2]
    yi = ups[i][3]
    xj = ups[j][2]
    yj = ups[j][3]
    
    return np.sqrt(pow(xi-xj,2)+pow(yi-yj,2))

def distance_time(i, j, ups, data=None):
    time_i = ups[i][0]
    time_j = ups[j][0]
    
    return np.sqrt(pow(time_i-time_j,2))

def distance_duration(i, j, ups, durations):
    return dist_duration(durations[i], durations[j])

def distance_color(i, j, ups, conflicts):
    color_i = ups[i][4]
    color_j = ups[j][4]
    
    if color_i == color_j:
        return 0
    else:
        max_up = len(ups)
        dist = 0
        
        conf_i = []
        if conflicts[i][0] <= max_up:
            conf_i.append(ups[conflicts[i][0]][4])
            
        if conflicts[i][1] <= max_up:
            conf_i.append(ups[conflicts[i][1]][4])
        
        conf_j = []
        if conflicts[j][0] <= max_up:
            conf_j.append(ups[conflicts[j][0]][4])
            
        if conflicts[j][1] <= max_up:
            conf_j.append(ups[conflicts[j][1]][4])
        
        if color_i in conf_j:
            dist = dist + 1
            
        if color_j in conf_i:
            dist = dist + 1
        
        return dist
    
def distance_user_embedding(i, j, ups, data):
    user_i = ups[i][1]
    user_j = ups[j][1]
    user_i_id = data['index'][user_i]
    user_j_id = data['index'][user_j]
    
    return np.linalg.norm(data['emb'][user_i_id]-data['emb'][user_j_id])

def distance_user_colors(i, j, ups, data):
    user_i = ups[i][1]
    user_j = ups[j][1]
    user_i_id = data['index'][user_i]
    user_j_id = data['index'][user_j]
    
    return (1.-data['emb'][user_i_id].todense() * data['emb'][user_j_id].todense().T)[0,0]

In [34]:
conflicts = compute_update_conflicts(ups)
durations = compute_update_durations(ups)
user_color, user_index_color = compute_user_color(ups)

#TODO: We are currently using 40 dimensions, we might need more
# We also need to understand whether these other parameters matter.
ndim=40
threshold=10
total_samples=200
n_negatives=5
n_iterations=10
user_index, emb = embed_users(G, ups, ndim, threshold, total_samples, n_negatives, n_iterations)

features = [{'name': "different_color", 'func': different_color, 'data': None}, 
    {'name': "distance_space",  'func': distance_space, 'data': None}, 
    {'name': "distance_time", 'func': distance_time, 'data': None}, 
    {'name': "distance_duration", 'func': distance_duration, 'data': durations}, 
    {'name': "distance_color", 'func': distance_color, 'data': conflicts},
    {'name': "distance_user_embedding", 'func': distance_user_embedding, 'data': {'index': user_index, 'emb': emb}},
    {'name': "distance_user_colors", 'func': distance_user_colors, 'data': {'index': user_index_color, 'emb': user_color}}]

python ../../signet/signet.py -l signet_id.txt -i signet.txt -o signet -d 40 -t 10 -s 200
reddit_place_project
signet

avg pos =  0.6544713490067806 , n =  3363278
avg neg =  1.5683112641071626 , n =  292678


In [35]:
pfile = open(file_prefix + 'features.pkl', 'wb')
pickle.dump(features, pfile)
pfile.close()

In [36]:
pfile = open(file_prefix + 'features.pkl', 'rb')
features = pickle.load(pfile)
pfile.close()

In [37]:
locations = store_locations("../data/atlas_complete.json")
folds = create_folds(min_x, min_y, max_x, max_y)

# List of dictionaries containing min_x, max_x, min_y, max_y for each fold
fold_boundaries = []
for fold in folds:
    fold_boundaries.append(get_fold_border(fold))

# All edges that belong to the validation fold need to be excluded
A_train, b_train = build_feat_label_data(G.unique_edges_file_name, ups, features, fold_boundaries=fold_boundaries, excluded_folds=[0])

In [38]:
# # Trying standard scaler 
# scaler_A = StandardScaler()
# scaler_b = StandardScaler()

# scaler_A.fit(A_train)
# scaled_A = scaler_A.transform(A_train)

# transformed_b = np.matrix(b_train).T
# scaler_b.fit(transformed_b)
# scaled_b = scaler_b.transform(transformed_b)
# scaled_b = scaled_b.T[0]

# print(scaled_b.shape)
# print(scaled_A.shape)
# print(b_train.shape)
# print(A_train.shape)

In [39]:
# trying minmax scaler
scaler_A = MinMaxScaler(feature_range=(0, 1))
scaler_b = MinMaxScaler(feature_range=(0, 1))

scaler_A.fit(A_train)
scaled_A = scaler_A.transform(A_train)

transformed_b = np.matrix(b_train).T
scaler_b.fit(transformed_b)
scaled_b = scaler_b.transform(transformed_b)
scaled_b = scaled_b.T[0]

print(scaled_b.shape)
print(scaled_A.shape)
print(b_train.shape)
print(A_train.shape)

(3081696,)
(3081696, 7)
(3081696,)
(3081696, 7)


In [40]:
scalerA_filename = "scaler_A.pkl"
scalerb_filename = "scaler_b.pkl"
pickle.dump(scaler_A, open(scalerA_filename, 'wb'))
pickle.dump(scaler_b, open(scalerb_filename, 'wb'))

In [41]:
model_gboost = GradientBoostingRegressor(random_state=1, n_estimators=25).fit(scaled_A, scaled_b)

In [42]:
model_nn = createNonlinearRegressionNeuralNet(scaled_A, scaled_b, train_proportion = 0.8, first_nodes=64, second_nodes=32)

Train on 2465356 samples, validate on 616340 samples
Epoch 1/256
 - 2318s - loss: 0.0368 - val_loss: 0.0261
Epoch 2/256
 - 2275s - loss: 0.0369 - val_loss: 0.0258
Epoch 3/256
 - 2225s - loss: 0.0371 - val_loss: 0.0268
Epoch 4/256
 - 2250s - loss: 0.0376 - val_loss: 0.0261
Epoch 5/256
 - 2255s - loss: 0.0377 - val_loss: 0.0254
Epoch 6/256
 - 2235s - loss: 0.0379 - val_loss: 0.0265
Epoch 00006: early stopping


In [43]:
model_gboost.train_score_[-1]

0.032673418346232455

In [44]:
prediction = model_nn.predict(scaled_A)
print(prediction)

[[0.01008998]
 [0.00855952]
 [0.01171956]
 ...
 [0.10358842]
 [0.03706743]
 [0.01214495]]


In [45]:
count = 0
for i in prediction:
    if i < 0:
        count+=1
        
print(count)

5233


In [46]:
count = 0
for i in scaled_b:
    if i < 0:
        count+=1
        
print(count)

0


In [47]:
ground_truth = create_ground_truth(input_file, min_x=min_x, max_x=max_x, min_y=min_y, max_y=max_y, projects_to_remove=projects_to_remove)

fold_boundaries = []
for fold in folds:
    fold_boundaries.append(get_fold_border(fold))


In [ ]:
kappa = 0.25

compute_edge_weights_multithread(G, ups, model_gboost, features, 5, file_prefix = file_prefix, scalerX=scalerA_filename, scalerY=scalerb_filename)
G.sort_edges()



comp_assign = region_segmentation(G, ups, kappa)
regions, sizes = extract_regions(comp_assign)
num_correct_counter, num_assignments_made, precision, recall = evaluate(locations, regions, ups, ground_truth, threshold=0.3, min_x=fold_boundaries[0]["min_x"], max_x=fold_boundaries[0]["max_x"], min_y=fold_boundaries[0]["min_y"], max_y=fold_boundaries[0]["max_y"])

print("Recall:", recall)
print("Precision:", precision)

In [ ]:
compute_edge_weights(G, ups, model_nn, features, scalerX=scalerA_filename, scalerY=scalerb_filename)
G.sort_edges()



comp_assign = region_segmentation(G, ups, kappa)
regions, sizes = extract_regions(comp_assign)
num_correct_counter, num_assignments_made, precision, recall = evaluate(locations, regions, ups, ground_truth, threshold=0.3, min_x=fold_boundaries[0]["min_x"], max_x=fold_boundaries[0]["max_x"], min_y=fold_boundaries[0]["min_y"], max_y=fold_boundaries[0]["max_y"])

print("Recall:", recall)
print("Precision:", precision)